# Function Calling with Amazon Bedrock

Welcome to the Function Calling notebook! In this section, we'll learn how to get structured outputs from our LLM by using function calling (also known as tool use). This powerful feature allows us to get predictable, formatted responses that we can easily use in our applications.

## Setup

Let's start with our usual imports:

In [ ]:
import boto3
import json
import math

session = boto3.Session()
bedrock = session.client(service_name='bedrock-runtime')

print("✅ Setup complete!")

## Basic Function Calling

Function calling is done through the converse API by providing a toolSpec. This is a json object that gets passed into the API request. In practice, you'd want to export Pydantic models as json before passing them in or out to make function calling more maintainable. Nobody likes passing around untyped JSON 😊

For now, we'll use JSON to get comfortable with function calling.

Let's start with a simple example. We'll create a function that can extract structured information from text:

In [ ]:
from typing import List, Dict, Any
# Define our tool specification
tool_list: List[Dict[str, Any]]= [
    {
        "toolSpec": {
            "name": "cosine",
            "description": "Calculate the cosine of x.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "x": {
                            "type": "number",
                            "description": "The number to pass to the function."
                        }
                    },
                    "required": ["x"]
                }
            }
        }
    }
]

SYSTEM_PROMPT: str = '''You are a helpful assistant. You must only do math by using a tool.'''

PROMPT_TEXT: str = '''What is the cosine of 7?'''

# Create our message
user_message: Dict[str, any] = { "role": "user", "content": [{ "text": PROMPT_TEXT }] }


MODEL_ID: str = "us.anthropic.claude-3-5-haiku-20241022-v1:0"

# Send request to Claude Haiku 3.5 via Bedrock
response: Dict[str, Any] = bedrock.converse(
    modelId=MODEL_ID,  # Using Sonnet 3.5 
    messages=[user_message],
    system=[{"text": SYSTEM_PROMPT}],
    inferenceConfig={ 'temperature': .1 },
    toolConfig={"tools": tool_list},
)

# Print the response
print(json.dumps(response['output']['message'], indent=2))

Notice how the model automatically extracts the information into the structured format we defined!

## More Complex Function Calling

Now let's try something more complex - we'll create a calculator that can handle different operations:

In [ ]:
# Define our calculator tools
calculator_tools = [
    {
        "toolSpec": {
            "name": "calculate",
            "description": "Perform a mathematical calculation.",
            "inputSchema": {
                "json": {
                    "type": "object",
                    "properties": {
                        "operation": {
                            "type": "string",
                            "enum": ["add", "subtract", "multiply", "divide"],
                            "description": "The mathematical operation to perform"
                        },
                        "x": {
                            "type": "number",
                            "description": "First number"
                        },
                        "y": {
                            "type": "number",
                            "description": "Second number"
                        }
                    },
                    "required": ["operation", "x", "y"]
                }
            }
        }
    }
]

def handle_calculation(tool_use):
    """Process the calculation request and return a result"""
    input_data = tool_use['input']
    x = input_data['x']
    y = input_data['y']
    
    if input_data['operation'] == 'add':
        result = x + y
    elif input_data['operation'] == 'subtract':
        result = x - y
    elif input_data['operation'] == 'multiply':
        result = x * y
    elif input_data['operation'] == 'divide':
        result = x / y if y != 0 else 'Error: Division by zero'
    
    return {
        "toolResult": {
            "toolUseId": tool_use['toolUseId'],
            "content": [{"json": {"result": result}}]
        }
    }

# Define the prompt text using XML tags for better Claude interaction
PROMPT_TEXT = """<instruction>Solve this math problem using the calculate tool. Show your reasoning in <thinking> tags and the final answer in <response> tags.</instruction>

<context>You have access to a calculate tool that can perform basic mathematical operations (add, subtract, multiply, divide).</context>

<question>What's fifteen times seven?</question>

Please structure your answer as follows:
<thinking>
[Show your reasoning about how to use the calculate tool]
</thinking>

<response>
[Provide the final calculated result]
</response>"""

# Try some calculations
messages = [
    {
        "role": "user",
        "content": [{"text": PROMPT_TEXT}]
    }
]

# First request to get the calculation request
response = bedrock.converse(
    modelId="anthropic.claude-3-sonnet-20240229-v1:0",
    messages=messages,
    inferenceConfig={"temperature": 0},
    toolConfig={"tools": calculator_tools},
    system=[{"text": "Use the calculate tool for all math operations."}]
)

# Get the tool use request
tool_use = response['output']['message']['content'][1]['toolUse']
print("Tool use request:")
print(json.dumps(tool_use, indent=2))

# Process the calculation
result_message = handle_calculation(tool_use)
messages.append(response['output']['message'])
messages.append({"role": "user", "content": [result_message]})

# Get the final response
final_response = bedrock.converse(
    modelId="anthropic.claude-3-sonnet-20240229-v1:0",
    messages=messages,
    inferenceConfig={"temperature": 0},
    toolConfig={"tools": calculator_tools},
    system=[{"text": "Use the calculate tool for all math operations."}]
)

print("\nFinal response:")
print(json.dumps(final_response['output']['message'], indent=2))


## Exercise

Now it's your turn! Try creating a tool that can parse dates in different formats. Here's a template to get you started:

1. Create a tool specification that accepts a date string and returns a structured format with:
   - year
   - month
   - day
2. Test it with different date formats like:
   - "March 15, 2024"
   - "2024-03-15"
   - "15/03/24"

Remember: The tool specification helps the model understand exactly what format you want the output in!